In [145]:
!pip install pyspark

In [146]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from sklearn.metrics import accuracy_score

# Reading the Dataset

In [150]:
spark = SparkSession.builder.appName('musicstreaming').getOrCreate()
df = spark.read.csv('music_streaming.csv', header = True, inferSchema = True)

# data cleaning & engineering

In [108]:
df.printSchema()

root
 |-- Artist Name: string (nullable = true)
 |-- Track Name: string (nullable = true)
 |-- Popularity: string (nullable = true)
 |-- danceability: string (nullable = true)
 |-- energy: string (nullable = true)
 |-- key: integer (nullable = true)
 |-- loudness: double (nullable = true)
 |-- mode: double (nullable = true)
 |-- speechiness: double (nullable = true)
 |-- acousticness: double (nullable = true)
 |-- instrumentalness: double (nullable = true)
 |-- liveness: double (nullable = true)
 |-- valence: double (nullable = true)
 |-- tempo: double (nullable = true)
 |-- duration_in min/ms: double (nullable = true)
 |-- time_signature: double (nullable = true)
 |-- Genre: double (nullable = true)



In [109]:
df.show()

+--------------------+--------------------+----------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+------------------+--------------+-----+
|         Artist Name|          Track Name|Popularity|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|duration_in min/ms|time_signature|Genre|
+--------------------+--------------------+----------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+------------------+--------------+-----+
|          Bruno Mars|That's What I Lik...|        60|       0.854| 0.564|  1|  -4.964| 1.0|     0.0485|      0.0171|            NULL|  0.0849|  0.899|134.071|          234596.0|           4.0|  5.0|
|              Boston|        Hitch a Ride|        54|       0.382| 0.814|  3|   -7.23| 1.0|     0.0406|      0.0011|         0.00401|   0.101|  0.569|116.454|          251733.0|           4.0| 10.0|


In [110]:
#drop missing values
df = df.na.drop()

In [111]:
#drop duplicates
df = df.dropDuplicates()

In [112]:
df.dtypes

[('Artist Name', 'string'),
 ('Track Name', 'string'),
 ('Popularity', 'string'),
 ('danceability', 'string'),
 ('energy', 'string'),
 ('key', 'int'),
 ('loudness', 'double'),
 ('mode', 'double'),
 ('speechiness', 'double'),
 ('acousticness', 'double'),
 ('instrumentalness', 'double'),
 ('liveness', 'double'),
 ('valence', 'double'),
 ('tempo', 'double'),
 ('duration_in min/ms', 'double'),
 ('time_signature', 'double'),
 ('Genre', 'double')]

# Queries (SparkSQL)


In [113]:
# Register DataFrame as a table for Spark SQL
df.createOrReplaceTempView("songs")

In [114]:
# a) Which genre has the highest average popularity?
query_a_sql = """
SELECT Genre, AVG(CAST(Popularity AS DOUBLE)) AS avg_popularity
FROM songs
GROUP BY Genre
ORDER BY avg_popularity DESC
LIMIT 1
"""
result_a_sql = spark.sql(query_a_sql)
result_a_sql.show()

+-----+------------------+
|Genre|    avg_popularity|
+-----+------------------+
|  4.0|55.140845070422536|
+-----+------------------+



In [115]:
# b) Display which artists have recorded the most number of songs with a duration of more than 5 minutes (Spark SQL)
query_b_sql = """
SELECT `Artist Name`, COUNT(*) AS num_songs
FROM songs
WHERE `duration_in min/ms` > 5
GROUP BY `Artist Name`
ORDER BY num_songs DESC
LIMIT 1
"""

result_b_sql = spark.sql(query_b_sql)
result_b_sql.show()

+------------------+---------+
|       Artist Name|num_songs|
+------------------+---------+
|The Rolling Stones|       26|
+------------------+---------+



In [116]:
# c) How many songs are included in every Genre? (Spark SQL)
query_c_sql = """
SELECT Genre, COUNT(*) AS num_songs
FROM songs
GROUP BY Genre
ORDER BY num_songs DESC
"""

result_c_sql = spark.sql(query_c_sql)
result_c_sql.show()

+-------+---------+
|  Genre|num_songs|
+-------+---------+
|   10.0|     2919|
|    6.0|     1783|
|    8.0|     1397|
|    1.0|      958|
|    2.0|      892|
|    9.0|      845|
|    5.0|      420|
|    7.0|      372|
|    0.0|      362|
|    3.0|      247|
|    4.0|      142|
|117.017|        1|
+-------+---------+



In [117]:
# d) Which artists dominated the charts? (Spark SQL)
query_d_sql = """
SELECT `Artist Name`, COUNT(*) AS num_songs
FROM songs
GROUP BY `Artist Name`
ORDER BY num_songs DESC
LIMIT 5
"""

result_d_sql = spark.sql(query_d_sql)
result_d_sql.show()

+------------------+---------+
|       Artist Name|num_songs|
+------------------+---------+
|The Rolling Stones|       28|
|    Britney Spears|       27|
|                U2|       25|
|         Metallica|       24|
|      Led Zeppelin|       22|
+------------------+---------+



In [118]:
# e) Recommend at least 5 fun/not-boring songs that can be played at a party (Spark SQL)
query_e_sql = """
SELECT `Artist Name`, `Track Name`, energy, danceability, valence
FROM songs
ORDER BY (energy + danceability + valence) DESC
LIMIT 5
"""

result_e_sql = spark.sql(query_e_sql)
result_e_sql.show()

+---------------+--------------------+------+------------+-------+
|    Artist Name|          Track Name|energy|danceability|valence|
+---------------+--------------------+------+------------+-------+
|LCD Soundsystem|Daft Punk Is Play...| 0.856|       0.921|  0.976|
|  Isaiah Rashad|Lay Wit Ya (feat....| 0.859|       0.925|  0.967|
|    Killer Mike|          Kryptonite| 0.916|       0.863|  0.964|
|        Shampoo|             Trouble| 0.971|       0.796|  0.963|
|   Prince Rapid|Gladiator - Instr...| 0.972|        0.78|  0.958|
+---------------+--------------------+------+------------+-------+



# Queries (SparkDataframes)

In [119]:
from pyspark.sql import functions as F

In [120]:
# a) Which genre has the highest average popularity?
result_a_df = df.groupBy("Genre").agg(F.avg("Popularity").alias("avg_popularity")).orderBy(F.desc("avg_popularity")).limit(1)
result_a_df.show()

+-----+------------------+
|Genre|    avg_popularity|
+-----+------------------+
|  4.0|55.140845070422536|
+-----+------------------+



In [121]:
# b) Display which artists have recorded the most number of songs with a duration of more than 5 minutes
result_b_df = df.filter(df["duration_in min/ms"] > 5).groupBy("Artist Name").agg(F.count("*").alias("num_songs")).orderBy(F.desc("num_songs")).limit(1)
result_b_df.show()

+------------------+---------+
|       Artist Name|num_songs|
+------------------+---------+
|The Rolling Stones|       26|
+------------------+---------+



In [122]:
# c) How many songs are included in every Genre?
result_c_df = df.groupBy("Genre").agg(F.count("*").alias("num_songs")).orderBy(F.desc("num_songs"))
result_c_df.show()

+-------+---------+
|  Genre|num_songs|
+-------+---------+
|   10.0|     2919|
|    6.0|     1783|
|    8.0|     1397|
|    1.0|      958|
|    2.0|      892|
|    9.0|      845|
|    5.0|      420|
|    7.0|      372|
|    0.0|      362|
|    3.0|      247|
|    4.0|      142|
|117.017|        1|
+-------+---------+



In [123]:
# d) Which artists dominated the charts?
result_d_df = df.groupBy("Artist Name").agg(F.count("*").alias("num_songs")).orderBy(F.desc("num_songs")).limit(5)
result_d_df.show()

+------------------+---------+
|       Artist Name|num_songs|
+------------------+---------+
|The Rolling Stones|       28|
|    Britney Spears|       27|
|                U2|       25|
|         Metallica|       24|
|      Led Zeppelin|       22|
+------------------+---------+



In [124]:
# e) Recommend at least 5 fun/not-boring songs that can be played at a party
result_e_df = df.select("Artist Name", "Track Name", "energy", "danceability", "valence").orderBy((df["energy"] + df["danceability"] + df["valence"]), ascending=False).limit(5)
result_e_df.show()

+---------------+--------------------+------+------------+-------+
|    Artist Name|          Track Name|energy|danceability|valence|
+---------------+--------------------+------+------------+-------+
|LCD Soundsystem|Daft Punk Is Play...| 0.856|       0.921|  0.976|
|  Isaiah Rashad|Lay Wit Ya (feat....| 0.859|       0.925|  0.967|
|    Killer Mike|          Kryptonite| 0.916|       0.863|  0.964|
|        Shampoo|             Trouble| 0.971|       0.796|  0.963|
|   Prince Rapid|Gladiator - Instr...| 0.972|        0.78|  0.958|
+---------------+--------------------+------+------------+-------+



# SparkML

In [133]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer


In [134]:
from pyspark.sql.functions import col

# Define the list of string columns to be cast to double
string_columns = ["Artist Name", "Track Name", "Popularity", "danceability", "energy"]

# Loop through each string column and cast it to double
for column in string_columns:
    df = df.withColumn(column, col(column).cast("double"))

In [135]:
# Split data into training and testing sets
train_data, test_data = df.randomSplit([0.8, 0.2], seed=42)

In [136]:
# Define features (excluding the target variable "Genre_index")
feature_columns = df.columns[:-1]


In [137]:
# Create separate instances of VectorAssembler for each pipeline
vector_assembler_lr = VectorAssembler(inputCols=feature_columns, outputCol="features")
vector_assembler_rf = VectorAssembler(inputCols=feature_columns, outputCol="features")
vector_assembler_gbt = VectorAssembler(inputCols=feature_columns, outputCol="features")


In [138]:
# Define classifiers
logistic_regression = LogisticRegression(labelCol="Genre", featuresCol="features")
random_forest = RandomForestClassifier(labelCol="Genre", featuresCol="features", numTrees=10)
gbt = GBTClassifier(labelCol="Genre", featuresCol="features", maxIter=10)


In [139]:
# Create pipelines for each classifier
pipeline_lr = Pipeline(stages=[vector_assembler_lr, logistic_regression])
pipeline_rf = Pipeline(stages=[vector_assembler_rf, random_forest])
pipeline_gbt = Pipeline(stages=[vector_assembler_gbt, gbt])


In [140]:
# Fit the pipelines on the training data
model_lr = pipeline_lr.fit(train_data)
model_rf = pipeline_rf.fit(train_data)
model_gbt = pipeline_gbt.fit(train_data)

Py4JJavaError: An error occurred while calling o1043.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 216.0 failed 1 times, most recent failure: Lost task 0.0 in stage 216.0 (TID 130) (8165a6e64508 executor driver): org.apache.spark.SparkException: [FAILED_EXECUTE_UDF] Failed to execute user defined function (`VectorAssembler$$Lambda$4308/0x000000084155e040`: (struct<Artist Name:double,Track Name:double,Popularity:double,danceability:double,energy:double,key_double_VectorAssembler_edb5faf10456:double,loudness:double,mode:double,speechiness:double,acousticness:double,instrumentalness:double,liveness:double,valence:double,tempo:double,duration_in min/ms:double,time_signature:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>).
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:198)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUse

In [ ]:

# Make predictions on the testing data
predictions_lr = model_lr.transform(test_data)
predictions_rf = model_rf.transform(test_data)
predictions_gbt = model_gbt.transform(test_data)


In [ ]:
# Evaluate the accuracy of each classifier
evaluator = MulticlassClassificationEvaluator(labelCol="Genre", predictionCol="prediction", metricName="accuracy")
accuracy_lr = evaluator.evaluate(predictions_lr)
accuracy_rf = evaluator.evaluate(predictions_rf)
accuracy_gbt = evaluator.evaluate(predictions_gbt)

In [ ]:
# Compare the accuracies of the classifiers
print("Accuracy - Logistic Regression: ", accuracy_lr)
print("Accuracy - Random Forest: ", accuracy_rf)
print("Accuracy - Gradient Boosted Tree: ", accuracy_gbt)

In [ ]:
# Determine the best classifier based on the results
best_classifier = max(accuracy_lr, accuracy_rf, accuracy_gbt)
if best_classifier == accuracy_lr:
    print("The best classifier is Logistic Regression.")
elif best_classifier == accuracy_rf:
    print("The best classifier is Random Forest.")
else:
    print("The best classifier is Gradient Boosted Tree.")

In [142]:
# Import necessary libraries
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier
from pyspark.ml import Pipeline
from pyspark.sql.functions import col

# Create a Spark session
spark = SparkSession.builder \
    .appName("MusicGenreClassification") \
    .getOrCreate()



# Cast string columns to double
string_columns = ["Artist Name", "Track Name", "Popularity", "danceability", "energy"]
for column in string_columns:
    df = df.withColumn(column, col(column).cast("double"))

# Split data into training and testing sets
train_data, test_data = df.randomSplit([0.8, 0.2], seed=42)

# Define features (excluding the target variable "Genre")
feature_columns = df.columns[:-1]

# Create VectorAssembler
vector_assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

# Define classifiers
logistic_regression = LogisticRegression(labelCol="Genre", featuresCol="features")
random_forest = RandomForestClassifier(labelCol="Genre", featuresCol="features", numTrees=10)
gbt = GBTClassifier(labelCol="Genre", featuresCol="features", maxIter=10)

# Create pipelines for each classifier
pipeline_lr = Pipeline(stages=[vector_assembler, logistic_regression])
pipeline_rf = Pipeline(stages=[vector_assembler, random_forest])
pipeline_gbt = Pipeline(stages=[vector_assembler, gbt])

# Fit the pipelines on the training data
model_lr = pipeline_lr.fit(train_data)
model_rf = pipeline_rf.fit(train_data)
model_gbt = pipeline_gbt.fit(train_data)

# Once the models are trained, you can use them for prediction on test data or further evaluation.


Py4JJavaError: An error occurred while calling o1138.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 220.0 failed 1 times, most recent failure: Lost task 0.0 in stage 220.0 (TID 133) (8165a6e64508 executor driver): org.apache.spark.SparkException: [FAILED_EXECUTE_UDF] Failed to execute user defined function (`VectorAssembler$$Lambda$4308/0x000000084155e040`: (struct<Artist Name:double,Track Name:double,Popularity:double,danceability:double,energy:double,key_double_VectorAssembler_b8cdbab6399f:double,loudness:double,mode:double,speechiness:double,acousticness:double,instrumentalness:double,liveness:double,valence:double,tempo:double,duration_in min/ms:double,time_signature:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>).
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:198)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUse

In [149]:

numericCols = ["Artist Name", "Track Name", "Popularity", "danceability", "energy"]

featurizationPipeline = Pipeline(stages = [VectorAssembler(inputCols=numericCols, outputCol="feature_vector")])

featurizationPipelineModel = featurizationPipeline.fit(df)
df = featurizationPipelineModel.transform(df)
train, test = df.randomSplit([0.8, 0.2], seed = 2018)

IllegalArgumentException: Data type string of column Artist Name is not supported.
Data type string of column Track Name is not supported.
Data type string of column Popularity is not supported.
Data type string of column danceability is not supported.
Data type string of column energy is not supported.

In [144]:
# Logistic Regression Model
lr = LogisticRegression(featuresCol = 'feature_vector', labelCol = 'Genre', maxIter=10)
lrModel = lr.fit(train)

# Make predictions on the test set.
predictions = lrModel.transform(test)

Py4JJavaError: An error occurred while calling o1219.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 224.0 failed 1 times, most recent failure: Lost task 0.0 in stage 224.0 (TID 136) (8165a6e64508 executor driver): org.apache.spark.SparkException: [FAILED_EXECUTE_UDF] Failed to execute user defined function (`VectorAssembler$$Lambda$4308/0x000000084155e040`: (struct<Artist Name:double,Track Name:double,Popularity:double,danceability:double,energy:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>).
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:198)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage2.hashAgg_doAggregateWithKeysOutput_0$(Unknown Source)
	at org.apache.spark.sql.catalys